In [2]:
import pandas as pd

ohiedf = pd.read_csv('ohie_data.csv')

### What are the actual treatment effects? 

In [9]:
#Ground truth treatment effect (in thresholded outcomes cutoff at 15)
ohiedf[(ohiedf['D'] == 1)]['Y'].mean()-ohiedf[(ohiedf['D'] == 0)]['Y'].mean()

-0.003408385422284027

In [12]:
#Ground truth treatment effect (in raw scores on 20-point continuous assessment)
ohiedf[(ohiedf['D'] == 1)]['phqtot_inp'].mean()-ohiedf[(ohiedf['D'] == 0)]['phqtot_inp'].mean()

-0.2743940081743297